In [1]:
import os
os.chdir('/content/drive/MyDrive/Machine_learning/Internship/ML/Text_summarization/Text_summarization_using_generative_models')

Introduction: In this notebook, you will go through a method of how you can use generative models to summarize large documents.

Install the required packeges: Vertex AI SDK, other packages and their dependencies

In [2]:
%pip install google-cloud-aiplatform PyPDF2 ratelimit backoff --upgrade --quiet --user

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
# Automatically restart the kernel after installing packages
# to ensure the environment has access to newly installed dependencies.

# Import the IPython module
import IPython

# Get the current IPython application instance
app = IPython.Application.instance()

# Forcefully restart the kernel to apply the changes
app.kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

**Authenticating your notebook environment**

In [2]:
from google.colab import auth # Using
auth.authenticate_user()

In [3]:
!gcloud auth login


You are running on a Google Compute Engine virtual machine.
It is recommended that you use service accounts for authentication.

You can run:

  $ gcloud config set account `ACCOUNT`

to switch accounts if necessary.

Your credentials may be visible to others with access to this
virtual machine. Are you sure you want to authenticate with
your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.

In [5]:
!gcloud beta billing projects describe tidal-datum-451517-i3


billingAccountName: billingAccounts/01C78A-DD26F7-10E17F
billingEnabled: true
name: projects/tidal-datum-451517-i3/billingInfo
projectId: tidal-datum-451517-i3


**Import libraries**

In [7]:
import vertexai

PROJECT_ID = "tidal-datum-451517-i3"  # Your Google Cloud Project ID
vertexai.init(project=PROJECT_ID, location="us-central1")  # Initialize Vertex AI in the specified region

In [8]:
from pathlib import Path #
import urllib
import warnings

import PyPDF2
import backoff
from google.api_core import exceptions
import ratelimit
from tqdm import tqdm
from vertexai.language_models import TextGenerationModel

warnings.filterwarnings("ignore")

**Import models**

In [9]:
#Load the pre-trained text generation model called text-bison
generation_model = TextGenerationModel.from_pretrained("text-bison")

**Preparing data files**

In [10]:
# To begin, it will be needed to download a pdf file for the summarizing tasks below.
# Define a folder to store the files
data_folder = "data"
Path(data_folder).mkdir(parents=True, exist_ok=True)

#  Define a pdf link to download and place to store the download file
pdf_url = "https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf"
pdf_file = Path(data_folder, pdf_url.split("/")[-1])

# Download the file using `urllib` library
urllib.request.urlretrieve(pdf_url, pdf_file)

(PosixPath('data/practitioners_guide_to_mlops_whitepaper.pdf'),
 <http.client.HTTPMessage at 0x7f526b89e390>)

In [11]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Print three pages from the pdf
for i in range(3):
    text = pages[i].extract_text().strip()
    print(f"Page {i}: {text} \n\n")

Page 0: Practitioners guide to MLOps:  
A framework for continuous 
delivery and automation of  
machine learning.White paper
May 2021
Authors:  
Khalid Salama,  
Jarek Kazmierczak,  
Donna Schut 


Page 1: Table of Contents
Executive summary  3
Overview of MLOps lifecycle and core capabilities  4
Deep dive of MLOps processes  15
Putting it all together  34
Additional resources  36Building an ML-enabled system  6
The MLOps lifecycle  7
MLOps: An end-to-end workflow  8
MLOps capabilities  9
      Experimentation  11
      Data processing  11
      Model training  11
      Model evaluation  12
      Model serving  12
      Online experimentation  13
      Model monitoring  13
      ML pipelines  13
      Model registry  14
      Dataset and feature repository  14
      ML metadata and artifact tracking  15
ML development  16
Training operationalization  18
Continuous training  20
Model deployment  23
Prediction serving  25
Continuous monitoring  26
Data and model management  29
      Dat

**Using Stuffing method**

The most straightforward method for providing data to a language model is by embedding it directly into the prompt as context without using memory. This involves incorporating all relevant details within the prompt, arranged in the desired sequence for the model to process.

In [12]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Entry string to concatenate all the extracted texts
concatenated_text = ""


# Loop through the pages
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Concat the extracted text to the concatenated text
    concatenated_text += text

print(f"There are {len(concatenated_text)} characters in the pdf")

100%|██████████| 37/37 [00:00<00:00, 79.09it/s]

There are 64758 characters in the pdf


In [23]:
!gcloud services list --enabled

NAME                                 TITLE
aiplatform.googleapis.com            Vertex AI API
analyticshub.googleapis.com          Analytics Hub API
artifactregistry.googleapis.com      Artifact Registry API
bigquery.googleapis.com              BigQuery API
bigqueryconnection.googleapis.com    BigQuery Connection API
bigquerydatapolicy.googleapis.com    BigQuery Data Policy API
bigquerymigration.googleapis.com     BigQuery Migration API
bigqueryreservation.googleapis.com   BigQuery Reservation API
bigquerystorage.googleapis.com       BigQuery Storage API
cloudapis.googleapis.com             Google Cloud APIs
cloudresourcemanager.googleapis.com  Cloud Resource Manager API
cloudtrace.googleapis.com            Cloud Trace API
compute.googleapis.com               Compute Engine API
datacatalog.googleapis.com           Google Cloud Data Catalog API
dataflow.googleapis.com              Dataflow API
dataform.googleapis.com              Dataform API
dataplex.googleapis.com              Cloud D

**create a prompt template that can be used later in the notebook**

In [24]:
prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.
    Return your response in bullet points which covers the key points of the text.

    ```{text}```

    BULLET POINT SUMMARY:
"""

**use LLM via the API to summarize the extracted texts**

In [25]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=concatenated_text)

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text
print(summary)

FailedPrecondition: 400 Project `164005685573` is not allowed to use Publisher Model `projects/tidal-datum-451517-i3/locations/us-central1/publishers/google/models/text-bison@002`

The model returned an error message: "400 Request contains an invalid argument" due to the extracted text exceeding the processing limit of the generative model.

To prevent this issue, you should input only a portion of the extracted text, such as the first 30,000 words.

In [26]:
# Define the prompt using the prompt template
prompt = prompt_template.format(text=concatenated_text[:30000])

# Use the model to summarize the text using the prompt
summary = generation_model.predict(prompt=prompt, max_output_tokens=1024).text

print(summary)

FailedPrecondition: 400 Project `164005685573` is not allowed to use Publisher Model `projects/tidal-datum-451517-i3/locations/us-central1/publishers/google/models/text-bison@002`

Summary

Since the full text exceeds the model's capacity, it is successfully generated a concise, bulleted summary of key information from a portion of the PDF. Below are the advantages and limitations of the stuffing method:
Advantages:

✔ Requires only a single model call.

✔ The model processes all the provided data at once, which can lead to a higher-quality summary.
Limitations:

✖ Most models have a context length limit, making this method ineffective for large documents or multiple documents.

✖ Suitable only for small text segments, as it cannot efficiently handle extensive content.

In the next session, alternative techniques will be exploreed to overcome the context length limitations of LLMs when dealing with longer texts.

Adding Rate Limit to Model Calls

When MapReduce or similar methods are used, multiple API calls are made to the model within a short period. However, there is a limit on the number of API calls allowed per minute, requiring a safety measure to be implemented in the code to prevent exceeding this limit. This ensures smooth execution and minimizes errors.

For this approach, the following steps are taken:

    A Python library called ratelimit is used to restrict the number of API calls per minute.
    A Python library called backoff is utilized to retry requests until the maximum time limit is reached.

The function below optimizes the API call process by restricting the number of requests to 20 per minute. Additionally, it incorporates a backoff mechanism that retries the API call upon encountering a "Resource Exhausted" exception. The waiting period increases exponentially until the five-minute limit is reached, after which retry attempts are discontinued.

In [27]:
CALL_LIMIT = 20  # Number of calls to allow within a period
ONE_MINUTE = 60  # One minute in seconds
FIVE_MINUTE = 5 * ONE_MINUTE


# A function to print a message when the function is retrying
def backoff_hdlr(details):
    print(
        "Backing off {} seconds after {} tries".format(
            details["wait"], details["tries"]
        )
    )


@backoff.on_exception(  # Retry with exponential backoff strategy when exceptions occur
    backoff.expo,
    (
        exceptions.ResourceExhausted,
        ratelimit.RateLimitException,
    ),  # Exceptions to retry on
    max_time=FIVE_MINUTE,
    on_backoff=backoff_hdlr,  # Function to call when retrying
)
@ratelimit.limits(  # Limit the number of calls to the model per minute
    calls=CALL_LIMIT, period=ONE_MINUTE
)
# This function will call the `generation_model.predict` function, but it will retry if defined exceptions occur.
def model_with_limit_and_backoff(**kwargs):
    return generation_model.predict(**kwargs)


Method 2: MapReduce

This approach involves dividing large data into smaller chunks and processing each chunk individually using a prompt. In summarization tasks, each chunk produces a summarized output. Once all the summaries are generated, a separate prompt is applied to merge them into a final summary.

Although this method is more complex than the previous one, it is often more effective for handling large datasets. To implement this approach, two prompt templates are prepared:

    One for generating summaries of individual chunks.
    Another for combining the summarized outputs.

In [28]:
initial_prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""

final_prompt_template = """
    Write a concise summary of the following text delimited by triple backquotes.
    Return your response in bullet points which covers the key points of the text.

    ```{text}```

    BULLET POINT SUMMARY:
"""


**Map step**

In this section, the PDF file will be reloaded, and the model will generate a summary for each page separately using the initial prompt template.

In [29]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries
initial_summary = []

# Iterate over the pages and generate a summary for each page
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Create a prompt for the model using the extracted text and a prompt template
    prompt = initial_prompt_template.format(text=text)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)


  0%|          | 0/37 [00:00<?, ?it/s]


FailedPrecondition: 400 Project `164005685573` is not allowed to use Publisher Model `projects/tidal-datum-451517-i3/locations/us-central1/publishers/google/models/text-bison@002`



Take a look at the first few summaries of from the initial Map phrase.


In [30]:
print("\n\n".join(initial_summary[:10]))

The number of characters in the initial summary will be counted to determine if it fits within the prompt's limitations.

In [31]:
len("\n".join(initial_summary))

0

Since a prompt previously accommodated 30,000 characters, this summary, having fewer characters, can also be input directly into a prompt. This will be done in the next step.

**Reduce Step**

A reduce function will be created to concatenate the summaries generated in the initial summarization step (Map step) and then apply the final prompt template to produce a more concise summary.

In [32]:
# Define a function to create a summary of the summaries


def reduce(initial_summary, prompt_template):
    # Concatenate the summaries from the inital step
    concat_summary = "\n".join(initial_summary)

    # Create a prompt for the model using the concatenated text and a prompt template
    prompt = prompt_template.format(text=concat_summary)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    return summary

Now, the next step involves **combining all summaries** into a more concise version using the **final prompt template** and the previously created function.

In [33]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

FailedPrecondition: 400 Project `164005685573` is not allowed to use Publisher Model `projects/tidal-datum-451517-i3/locations/us-central1/publishers/google/models/text-bison@002`

Recap

The entire paper was successfully summarized into a few bullet points using the MapReduce method. Below are its advantages and limitations:
Pros:

✔ Capable of summarizing large documents efficiently.
✔ Supports parallel processing, as each page is summarized independently.
Cons:

✖ Requires multiple model calls, increasing computational cost.
✖ Loss of context between pages since each is processed separately.

### **Next Section**  

In the following section, an alternative method will be explored, which processes **multiple chunks (pages) per prompt** to generate a more comprehensive summary.

**Method 3: MapReduce with Overlapping Chunks**

### **Overview of the Method: Overlapping Chunks**  

This approach is similar to **MapReduce** but introduces a key improvement: **overlapping chunks**. Instead of summarizing each page independently, multiple pages are grouped and summarized together. This technique helps **preserve context** and **retain more information between chunks**, leading to **more accurate summaries**.  

However, combining multiple chunks may sometimes **exceed the model's token limit**. If this happens, possible solutions include:  
- **Using a chunk-splitting method** to divide the text more efficiently.  
- **Removing some initial chunks** strategically to stay within the token limit.

**Map Step**

In this section, the PDF file will be processed again, and the model will be used to summarize multiple pages together rather than individually. The initial prompt template defined earlier will be applied to generate summaries while preserving context across pages.

In [27]:
# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the extracted text from the pages
text_from_pages = []

# Iterate over the pages and generate a summary for each page
for page in tqdm(pages):
    # Extract the text from the page and remove any leading or trailing whitespace
    text = page.extract_text().strip()

    # Append the extracted text to the list of extracted text
    text_from_pages.append(text)

100%|██████████| 37/37 [00:00<00:00, 87.58it/s]


 **Defining Chunk Size and Summarizing Chunks**  

In this step, the **chunk size** (number of pages to be combined per summary) will be specified. The model will then process and summarize each chunk using the predefined prompt template, ensuring that multiple pages are summarized together while maintaining context.

In [28]:
CHUNK_SIZE = 2  # number of overlapping pages

# Read the PDF file and create a list of pages
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries
initial_summary = []

# Iterate over the pages and generate a summary for a few pages as one chunk based on `CHUNK_SIZE`
for i in tqdm(range(len(pages))):
    # Select a list of pages to merge as one chunk
    pages_to_merge = [x for x in range(i, i + CHUNK_SIZE) if x < len(pages)]

    extracted_texts = [text_from_pages[x] for x in pages_to_merge]

    # Concatenate the
    text = "\n".join(extracted_texts)

    # Create a prompt for the model using the concatenated text and a prompt template
    prompt = initial_prompt_template.format(text=text)

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)

    # If the last page is reached, break the loop
    if pages_to_merge[-1] == len(reader.pages):
        break


  0%|          | 0/37 [00:00<?, ?it/s]


PermissionDenied: 403 Permission denied on resource project [my_summary_gen_ai]. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "[my_summary_gen_ai]"
}
metadata {
  key: "consumer"
  value: "projects/[my_summary_gen_ai]"
}
, locale: "en-US"
message: "Permission denied on resource project [my_summary_gen_ai]."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

In [29]:
print("\n\n".join(initial_summary[:10]))

### **Reduce Step**  

The next step involves **combining all generated summaries** into a more concise version using the **final prompt template** and the previously implemented function. This step further refines the summary while preserving key information.

In [ ]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

### **Recap**  

The model successfully summarized the entire paper into a few bullet points using the **MapReduce with Overlapping Chunks** method. Below are the advantages and limitations of this approach:  

#### **Pros:**  
✔ **Capable of summarizing large documents** efficiently.  
✔ **Preserves context** between pages by summarizing sequential pages together.  
✔ **Supports parallel processing**, as summaries are generated independently.  

#### **Cons:**  
✖ **Requires multiple model calls**, increasing processing overhead.  
✖ **Slightly slower** than the standard MapReduce method.  
✖ **Produces larger input text**, which may approach the model’s token limit.

### **Next Section**  

In the upcoming section, a different approach will be explored, where **only the summary from the previous page** is used instead of the full text. This method helps retain context across pages while reducing the input size for the model.

### **Method 4: MapReduce with Rolling Summary (Refine)**  

In some cases, summarizing multiple pages together may exceed the model’s token limit. To address this, a **rolling summary approach** will be used.  

Instead of summarizing chunks independently, this method takes **the summary from the previous step** along with the **next page** to generate a refined summary. This ensures that each prompt retains **context from the previous page**, leading to a more **coherent and accurate** final summary.

In [ ]:
initial_prompt_template = """
    Taking the following context delimited by triple backquotes into consideration:

    ```{context}```

    Write a concise summary of the following text delimited by triple backquotes.

    ```{text}```

    CONCISE SUMMARY:
"""


# Read the PDF file and create a list of pages.
reader = PyPDF2.PdfReader(pdf_file)
pages = reader.pages

# Create an empty list to store the summaries.
initial_summary = []

# Iterate over the pages and generate a summary
for idx, page in enumerate(tqdm(pages)):
    # Extract the text from the page and remove any leading or trailing whitespace.
    text = page.extract_text().strip()

    if idx == 0:  # if current page is the first page, no previous context
        prompt = initial_prompt_template.format(context="", text=text)

    else:  # if current page is not the first page, previous context is the summary of the previous page
        prompt = initial_prompt_template.format(
            context=initial_summary[idx - 1], text=text
        )

    # Generate a summary using the model and the prompt
    summary = model_with_limit_and_backoff(prompt=prompt, max_output_tokens=1024).text

    # Append the summary to the list of summaries
    initial_summary.append(summary)


### **Listing Initial Summary Entries**  

In this step, a few entries from the **initial summary list** will be displayed. This provides a reference for how the summaries have been generated so far and helps in understanding the progression of information before applying the rolling summary approach.

In [30]:
initial_summary[:10]

[]

Handling Duplicate Entries

Since the rolling summary approach carries over context from previous pages, some duplicate entries in the summary list are expected. These duplicates can be easily removed by using the set() function, which ensures that only unique summaries are retained.

In [31]:
initial_summary = set(initial_summary)  # set() function removes duplicate items

### **Reduce Step**  

The next step involves **combining all refined summaries** into a more concise version using the **final prompt template** and the previously implemented function. This step ensures that the summary remains **coherent and compact** while preserving key contextual information.

In [32]:
# Use defined `reduce` function to summarize the summaries
summary = reduce(initial_summary, final_prompt_template)

print(summary)

PermissionDenied: 403 Permission denied on resource project [my_summary_gen_ai]. [reason: "CONSUMER_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "[my_summary_gen_ai]"
}
metadata {
  key: "consumer"
  value: "projects/[my_summary_gen_ai]"
}
, locale: "en-US"
message: "Permission denied on resource project [my_summary_gen_ai]."
, links {
  description: "Google developers console"
  url: "https://console.developers.google.com"
}
]

### **Recap**  

The model successfully summarized the entire paper into a few bullet points using the **MapReduce with Rolling Summary** method. Below are the advantages and limitations of this approach:  

#### **Pros:**  
✔ **Capable of summarizing large documents** effectively.  
✔ **Preserves context** across pages by incorporating summaries from previous sections.  

#### **Cons:**  
✖ **Requires multiple model calls**, increasing processing time.  
✖ **Not suitable for parallel processing**, as each summary depends on the previous one.